## Test Parameters

In [1]:
username = 'alexl'
testid = 6536
tenure_grouping = 35
allocation_start = 20150723
allocation_end = 20150813

## Step 1: Manual Dashboard Prep

In [2]:
run_vhs_dump = 0

In [3]:
%%time
!python manual_dashboard_prep.py -userdb {username} -testid {testid} -tenure_grouping {tenure_grouping} \\
-allocation_start {allocation_start} -allocation_end {allocation_end} -run_vhs_dump {run_vhs_dump}

Traceback (most recent call last):
  File "manual_dashboard_prep.py", line 1, in <module>
    from query_launcher import query_launcher
  File "/Users/alexliu/Downloads/prodexp-master/manual_dashboard/sql/query_launcher.py", line 2, in <module>
    import kragle as kg
ImportError: No module named kragle
CPU times: user 2 ms, sys: 4.35 ms, total: 6.35 ms
Wall time: 138 ms


## Step 2: Custom SQL

### Allocation Filters

In [2]:
allocation_filters = []

name = "streamed_darwin"
query = """
SELECT DISTINCT account_id
FROM alexl.vhs_dump_for_manual_dashboard_6536
WHERE ui_version LIKE '%darwin%'
AND region_dateint > -1
"""
allocation_filters.append((query, name))

name = "cross_allocated_5935"
query = """
SELECT DISTINCT account_id
FROM dse.exp_allocation_denorm_f 
WHERE test_id = 5935
"""
allocation_filters.append((query, name))

### Activity Filters

In [3]:
activity_filters = []

name = "ui_version"
query = "CASE WHEN ui_version LIKE '%darwin%' THEN 'Darwin' ELSE 'Non-Darwin' END"
activity_filters.append((query, name))

name = "is_original"
query = "CASE WHEN is_original = 1 THEN 'Original' ELSE 'Licensed' END"
activity_filters.append((query, name))

name = "is_novel"
query = "CASE WHEN is_novel_play = 1 THEN 'Novel' ELSE 'Non-Novel' END"
activity_filters.append((query, name))

### Optionally merge cells

In the example below, we combine cells 2,3,4 into a new effective cell 10.

In [6]:
cell_mapping = {(2,3,4):10}

### Generate query

In [7]:
from manual_dashboard_generator import manual_dashboard_generator
dashboard = manual_dashboard_generator(username)

In [8]:
test_params = {
    'userdb':username,
    'testid':testid,
    'tenure_grouping':tenure_grouping
    }

dashboard.set_filters(allocation_filters, activity_filters, cell_mapping)
query = dashboard.generate_query(test_params)

In [9]:
print query

use mramm;

DROP TABLE IF EXISTS mramm.manual_dashboard_streaming_5846;
CREATE TABLE mramm.manual_dashboard_streaming_5846 AS
SELECT
    test_id,
    test_name,
    test_cell_nbr,
    test_cell_name AS name,
    -- Standard filters
    COALESCE(allocation_type, 'All Allocations') AS filter_allocation_type,
    COALESCE(signup_region, 'All Allocations') AS filter_signup_region,
    COALESCE(completed_activity_window, 'All Allocations') AS filter_completed_activity_window,
    -- Allocation filters
    COALESCE(CASE WHEN streamed_darwin.account_id IS NOT NULL THEN 1 ELSE 0 END, 'All Allocations') AS filter_streamed_darwin,
COALESCE(CASE WHEN cross_allocated_5935.account_id IS NOT NULL THEN 1 ELSE 0 END, 'All Allocations') AS filter_cross_allocated_5935,
    -- Activity Filters
    CASE WHEN vhs.ui_version IS NULL THEN 'No Streaming' ELSE vhs.ui_version END AS filter_ui_version,
CASE WHEN vhs.is_original IS NULL THEN 'No Streaming' ELSE vhs.is_original END AS filter_is_original,
CASE WHEN

In [8]:
dashboard.run_dashboard(email = True)

.......................................................................................................................................
